In [1]:
from IPython.display import display
import planner

plan_text = """\
Done; do something a 20m
"""
controller = planner.PlanController(plan_text, "23:30")
display(controller.container)

In [2]:
from IPython.display import display
import logger


log_controller = logger.LogController()
display(log_controller.container)